# side model

In [1]:
import numpy as np
import pandas as pd

from jesse.helpers import date_to_timestamp

from strategies.BinanceBtcEntropyBarV1.config import SIDE

df_features = pd.read_parquet("data/features.parquet")

features = df_features[SIDE]
label = np.load("data/side_label.npy")
len_gap = len(label) - len(df_features)
label = label[len_gap:]

print(features.shape)
print(label.shape)

mask = features.index < date_to_timestamp("2025-01-01")
features_masked = features[mask]
label_masked = label[mask]

print(features_masked.shape)
print(label_masked.shape)
print(np.unique(label_masked, return_counts=True))

features.isna().sum().sort_values(ascending=False)

(18718, 472)
(18718,)
(17386, 472)
(17386,)
(array([0, 1]), array([8729, 8657]))


williams_r                        0
trendflex_dt_lag3                 0
trendflex_dt_lag6                 0
adaptive_cci_lag19                0
norm_on_balance_volume_ddt        0
                                 ..
swamicharts_stochastic_8          0
evenbetter_sinewave_short_lag4    0
roofing_filter                    0
cmma_lag3                         0
cmma_lag17                        0
Length: 472, dtype: int64

In [2]:
import lightgbm as lgb

params = {
    "objective": "binary",
    "metric": "auc",
    "num_threads": -1,
    "verbose": -1,
    "is_unbalance": True,
    "extra_trees": False,
    "num_leaves": 100,
    "max_depth": 20,
    "min_gain_to_split": 1e-8,
    "min_data_in_leaf": 20,
    "lambda_l1": 1e-4,
    "lambda_l2": 1e-4,
}
dtrain = lgb.Dataset(features_masked, label_masked)
res = lgb.cv(params, dtrain, num_boost_round=100, nfold=5, stratified=True)
res.keys()

dict_keys(['valid auc-mean', 'valid auc-stdv'])

In [3]:
res["valid auc-mean"][-1]

0.9101366058458827

In [4]:
import lightgbm as lgb
import optuna


def objective(trial):
    METRIC = "auc"

    params = {
        "objective": "binary",
        "metric": METRIC,
        "num_threads": -1,
        "verbose": -1,
        "is_unbalance": trial.suggest_categorical("is_unbalance", [True, False]),
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 500),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 100),
    }
    dtrain = lgb.Dataset(features, label)
    # dtest = lgb.Dataset(side_features_test, side_label_test)
    model_res = lgb.cv(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1000),
        metrics=METRIC,
    )
    return model_res[f"valid {METRIC}-mean"][-1]


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

[I 2025-06-14 18:59:48,967] A new study created in memory with name: no-name-bbb46bd2-4eab-4ffc-9c34-37d902146886
[I 2025-06-14 18:59:53,390] Trial 0 finished with value: 0.8970507610328443 and parameters: {'is_unbalance': False, 'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 241, 'max_depth': 941, 'min_gain_to_split': 0.5293258877576078, 'min_data_in_leaf': 114, 'lambda_l1': 70.32574327704353, 'lambda_l2': 30.902749531909777, 'num_boost_round': 312}. Best is trial 0 with value: 0.8970507610328443.
[I 2025-06-14 19:00:00,406] Trial 1 finished with value: 0.8958810768794356 and parameters: {'is_unbalance': False, 'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 65, 'max_depth': 835, 'min_gain_to_split': 0.7414172751658835, 'min_data_in_leaf': 401, 'lambda_l1': 99.1168536560526, 'lambda_l2': 12.079914677993608, 'num_boost_round': 899}. Best is trial 0 with value: 0.8970507610328443.
[I 2025-06-14 19:00:06,607] Trial 2 finished with value: 0.9023317294735381 and parameters

In [5]:
print("side model long label: ")
study.best_params

side model long label: 


{'is_unbalance': True,
 'extra_trees': False,
 'boosting': 'gbdt',
 'num_leaves': 199,
 'max_depth': 590,
 'min_gain_to_split': 0.0022019736048242283,
 'min_data_in_leaf': 154,
 'lambda_l1': 5.447668983188831,
 'lambda_l2': 18.67952679836016,
 'num_boost_round': 818}

In [6]:
import lightgbm as lgb  # noqa

params = {
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

dtrain = lgb.Dataset(features_masked, label_masked)
side_model = lgb.train(params, dtrain)
side_model_prod = lgb.train(params, lgb.Dataset(features, label))

In [7]:
import plotly.express as px

test_features = features[features.index > date_to_timestamp("2025-01-01")]
res = side_model.predict(test_features)

fig = px.histogram(res, nbins=100)
fig.show()

In [8]:
side_model.save_model("strategies/BinanceBtcEntropyBarV1/model/model_side.txt")
side_model_prod.save_model(
    "strategies/BinanceBtcEntropyBarV1/model/model_side_prod.txt"
)

# meta model

In [1]:
import numpy as np
import pandas as pd

from jesse.helpers import date_to_timestamp

from strategies.BinanceBtcEntropyBarV1.config import (
    META_ALL,
    SIDE,
    get_side_model,
)

df_features = pd.read_parquet("data/features.parquet")
meta_label = np.load("data/label_meta.npy")

side_model = get_side_model(False)

side_model_res = side_model.predict(df_features[SIDE])

df_features["model"] = side_model_res

meta_features = df_features[META_ALL]
print(meta_features.shape)
print(meta_label.shape)

mask = meta_features.index < date_to_timestamp("2025-01-01")
meta_features_masked = meta_features[mask]
meta_label_masked = meta_label[mask]

print(meta_features_masked.shape)
print(meta_label_masked.shape)

meta_features.isna().sum(axis=0).sort_values(ascending=False)

(18718, 73)
(18718,)
(17386, 73)
(17386,)


corwin_schultz_estimator_lag1    0
acc_swing_index_ddt              0
natr_ddt_lag1                    0
natr_ddt_lag3                    0
price_change_oscillator          0
                                ..
amihud_lambda_dt                 0
amihud_lambda_dt_lag11           0
amihud_lambda_ddt_lag11          0
bandpass_dt_lag6                 0
model                            0
Length: 73, dtype: int64

In [2]:
import lightgbm as lgb
from sklearn.metrics import (
    f1_score,  # noqa
    fbeta_score,  # noqa
)

METRIC = "f1"


def eval_metric(preds, eval_dataset):
    metric_name = METRIC
    y_true = eval_dataset.get_label()
    value = f1_score(y_true, preds > 0.5, average="macro")
    higher_better = True
    return metric_name, value, higher_better


params = {
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    "is_unbalance": True,
    "extra_trees": False,
    "num_leaves": 100,
    "max_depth": 20,
    "min_gain_to_split": 1e-8,
    "min_data_in_leaf": 20,
    "lambda_l1": 1e-4,
    "lambda_l2": 1e-4,
}
dtrain = lgb.Dataset(meta_features, meta_label)
res = lgb.cv(
    params, dtrain, num_boost_round=100, nfold=5, stratified=True, feval=eval_metric
)
res.keys()

dict_keys(['valid binary_logloss-mean', 'valid binary_logloss-stdv', 'valid f1-mean', 'valid f1-stdv'])

In [3]:
import optuna


def objective(trial):
    params = {
        "objective": "binary",
        "is_unbalance": trial.suggest_categorical("is_unbalance", [True, False]),
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 500),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 100),
    }
    dtrain = lgb.Dataset(meta_features, meta_label)
    # dtest = lgb.Dataset(meta_features_test, meta_label_test)
    model_res = lgb.cv(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1000),
        stratified=True,
        feval=eval_metric,
    )
    return model_res[f"valid {METRIC}-mean"][-1]


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

[I 2025-06-14 20:30:12,274] A new study created in memory with name: no-name-d6292fed-197e-4bf2-8fce-dc3bd97ffe56
[I 2025-06-14 20:30:20,548] Trial 0 finished with value: 0.5833687201941417 and parameters: {'is_unbalance': True, 'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 142, 'max_depth': 584, 'min_gain_to_split': 0.3220703152794512, 'min_data_in_leaf': 171, 'lambda_l1': 70.61299781342453, 'lambda_l2': 89.37727955840664, 'num_boost_round': 782}. Best is trial 0 with value: 0.5833687201941417.
[I 2025-06-14 20:30:25,697] Trial 1 finished with value: 0.5886403373363983 and parameters: {'is_unbalance': True, 'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 117, 'max_depth': 922, 'min_gain_to_split': 0.8958701966698767, 'min_data_in_leaf': 110, 'lambda_l1': 55.037712593236016, 'lambda_l2': 34.73303014822538, 'num_boost_round': 489}. Best is trial 1 with value: 0.5886403373363983.
[I 2025-06-14 20:30:32,421] Trial 2 finished with value: 0.5987359352570708 and parameters: 

In [4]:
print("meta model: ")
study.best_params

meta model: 


{'is_unbalance': True,
 'extra_trees': False,
 'boosting': 'gbdt',
 'num_leaves': 382,
 'max_depth': 570,
 'min_gain_to_split': 0.11662508654243876,
 'min_data_in_leaf': 84,
 'lambda_l1': 21.27804015281982,
 'lambda_l2': 17.57573303502409,
 'num_boost_round': 155}

In [5]:
import lightgbm as lgb

params = {
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

model = lgb.train(
    params,
    lgb.Dataset(meta_features_masked, meta_label_masked),
)

prod_model = lgb.train(params, lgb.Dataset(meta_features, meta_label))

In [6]:
import plotly.express as px

res = model.predict(
    meta_features[meta_features.index > date_to_timestamp("2025-01-01")]
)
pred_label = (res > 0.5).astype(int)

fig = px.histogram(res, nbins=100)
fig.show()

In [7]:
model.save_model("strategies/BinanceBtcEntropyBarV1/model/model_meta.txt")
prod_model.save_model("strategies/BinanceBtcEntropyBarV1/model/model_meta_prod.txt")